In [11]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [12]:
# Import data
df = pd.read_pickle(r'data\04_fct\fct_demographic_offers_and_transactions.pkl')

In [16]:
### Segment Customers ###
# Extract demographic features for clustering
demographic_features = df[['age', 'income', 'days_as_member', 'gender_F', 'gender_M']]

# Standardize the features
scaler = StandardScaler()
demographic_features_scaled = scaler.fit_transform(demographic_features)

# Apply K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df['segment'] = kmeans.fit_predict(demographic_features_scaled)

# Display the first few rows with the segment labels
df[['age', 'income', 'days_as_member', 'gender_F', 'gender_M', 'segment']].head()


,age,income,days_as_member,gender_F,gender_M,segment
customer_id,,,,,,
0610b486422d4921ae7d2bf64640c50b,55,112000,376,1,0,0
0610b486422d4921ae7d2bf64640c50b,55,112000,376,1,0,0
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,0
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,0
78afa995795e4d85b5d9ceeca43f5fef,75,100000,443,1,0,0


In [ ]:
### Calculate Completion Rates by Offer Type and Segement ###
df_filtered = df[df['viewed_before_completion'] == 1]

# Group data by segment and offer attributes, then calculate the response rate
response_data = df.groupby(['segment', 'is_bogo', 'is_discount', 'reward', 'difficulty', 'duration_hrs']).agg({
    'offer_viewed': 'mean',
    'offer_completed': 'mean'
}).reset_index()

# Rename columns for clarity
response_data.rename(columns={'offer_viewed': 'viewed_rate', 'offer_completed': 'completion_rate'}, inplace=True)

# Identify the top segments for each offer type
top_segments = response_data.sort_values(by='completion_rate', ascending=False).groupby(['is_bogo', 'is_discount', 'reward', 'difficulty', 'duration_hrs']).head(1)

top_segments

,segment,is_bogo,is_discount,reward,difficulty,duration_hrs,viewed_rate,completion_rate
3,0,0,1,2,10,240,0.973333,0.861961
4,0,0,1,3,7,168,0.969817,0.837967
9,0,1,0,10,10,168,0.821319,0.776074
7,0,1,0,5,5,168,0.606583,0.774295
6,0,1,0,5,5,120,0.973258,0.766613
2,0,0,1,2,10,168,0.618267,0.753318
8,0,1,0,10,10,120,0.968069,0.729751
5,0,0,1,5,20,240,0.420155,0.720155
21,2,0,0,0,0,96,0.562092,0.000000
20,2,0,0,0,0,72,0.955869,0.000000


Conclusion
* Segment 0: Highly responsive to discount offers with lower rewards and higher difficulty over longer durations.
* Characteristics: Segment 0 consists of older males with an average age of 54.57, a mean income of $64,673, and a longer membership duration (mean of 1088.07 days).
* Segment 1: Highly responsive to high-reward BOGO offers with higher difficulty.
* Characteristics: Segment 1 consists of older females with an average age of 57.46, a mean income of $71,303, and a moderate membership duration (mean of 510.97 days).
* Segment 2: Highly responsive to both high-reward BOGO offers and high-difficulty discount offers over various durations.
* Characteristics: Segment 2 consists of middle-aged males with an average age of 50.95, a mean income of $59,464, and a shorter membership duration (mean of 261.18 days).